In [4]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
# !pip install nbconvert
from sqlalchemy import create_engine
import pandas as pd
import sys
import getpass
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import Counter
import numpy as np
from scipy import stats
import seaborn as sns

from sklearn.feature_selection import mutual_info_classif, GenericUnivariateSelect, SelectFromModel, RFE, RFECV



In [42]:
features = pd.read_excel('AFSZ.xlsx', sheet_name='Arkusz2')
features=features.iloc[0:38,:]
features.head()

,numer,imie,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,64
0,1,CZUBA,1,0,1,0,1,0/4,1,0/5,...,1,"0/4,4",1,0/3,1,"0/4,8",1,"0/3,6",1.0,0/3
1,1,BOCHENEK,1,0,1,0,1,0,1,"0/9,6",...,1,"0/5,2",1,"0/4,9",1,"0/2,7",1,0/2,1.0,0/2
2,1,MARCZYŃSKA,1,0,1,0,1,"0/6,2",1,0/4,...,1,"0/4,5",1,0/<3,1,0/<2,1,0/<2,1.0,0/<2
3,1,ZIELIŃSKA,1,0,1,0,1,"0/4,3",1,"0/3,4",...,1,0/<3,1,0/<3,1,0/<3,1,0/<3,1.0,0/<3
4,1,SERMET,1,0,1,0,1,"0/9,2",1,"0/3,9",...,1,"0/4,2",1,"0/3,3",1,"0/3,3",1,"0/2,5",1.0,0/<2


In [105]:
features_mean = features.groupby(features.iloc[:,0]).mean().copy()
features_std =  features.groupby(features.iloc[:,0]).std().copy()
features_median =  features.groupby(features.iloc[:,0]).median().copy()
features_75 = features.groupby(features.iloc[:,0]).quantile(0.75).copy()
features_25 = features.groupby(features.iloc[:,0]).quantile(0.25).copy()

In [106]:
features_75

,numer,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
numer,,,,,,,,,,,,,,,,,,,,,
1,1.0,3120.0,1920.0,2970.0,945.0,2772.0,1155.0,1620.0,1770.0,1212.75,...,0.0,0.0,0.0,2772.0,0.0,900.0,1237.5,1386.0,0.0,0.0
2,2.0,480.0,5880.0,2920.5,1440.0,1138.5,330.0,1440.0,1350.0,742.50,...,0.0,49.5,0.0,1386.0,0.0,840.0,540.0,693.0,0.0,120.0
3,3.0,240.0,560.0,2970.0,720.0,2772.0,330.0,2880.0,3780.0,1386.00,...,0.0,1980.0,0.0,2079.0,0.0,1440.0,3600.0,2079.0,0.0,0.0


In [199]:
feature_vector = features_mean.merge(features_std, left_index=True, right_index=True, suffixes=('','_std'))
feature_quantile = features_75.merge(features_25, left_index=True, right_index=True, suffixes=('_75','_25'))
feature_quantile_combined = feature_quantile.merge(feature_vector,left_index=True, right_index=True)
feature_combine = features_median.merge(feature_quantile_combined, left_index=True, right_index=True, suffixes=('_median','_mean'))
#feature_vector.head()
feature_combine.head()
feature_combine.reset_index(inplace=True)


In [203]:
feature_combine

,participant_ID,participant_ID_median,kol.C._median,odst.czasu M-C_median,palenie przed 1_median,palenie przed 2_median,palenie przed 3_median,palenie I 1_median,palenie I 2_median,palenie I 3_median,...,6+4cm 3 L_std,7+4cm 3 L_std,8+4cm 3 L_std,9+4cm 3 L_std,OBJĘTOŚĆ L1_std,OBJĘTOŚĆ L2_std,OBJĘTOŚĆ L3_std,MASA CIAŁA (kg)_std,WZROST (m)_std,BMI_std
0,1.0,1.0,2.0,18.0,0.0,0.0,1.0,0.0,0.0,1.0,...,2.990378,3.277947,3.541923,3.699573,502.639876,510.239935,735.737444,11.125149,0.059033,2.938653
1,2.0,2.0,2.0,8.0,0.0,0.0,1.0,0.0,0.0,1.0,...,6.899776,6.197712,5.997393,5.841809,908.207047,952.944990,925.439007,24.046262,0.051275,8.369742
2,3.0,3.0,2.0,21.0,0.0,0.0,1.0,0.0,0.0,1.0,...,3.254535,3.781534,3.630702,3.266803,602.534034,699.026790,641.893938,14.339735,0.029496,5.837216


In [201]:
export_excel = feature_combine.to_excel (r'C:\Users\perszing\Documents\ola\statystykasredniemediana0808.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path


In [13]:
features = pd.read_excel('23.07 Kwestionariusze(1).xlsx', sheet_name='Arkusz7')
#features=features.iloc[0:38,:]
#features=features.iloc[0:114,:]
#features.head()
filtered_cols = list(features.columns)
features

,numer,średnia_pom1,odchylenie std_pom1,mediana_pom1
0,1,1370.133971,2133.473943,540
1,2,778.271531,1551.351870,0
2,3,468.171463,937.480629,0
3,4,1347.805785,1947.567460,600
4,5,879.537190,1648.851999,90
5,6,497.057851,986.111508,0
6,7,1307.867769,2186.292333,495
7,8,457.706612,1071.425903,0
8,9,323.745833,624.358464,0
9,10,1605.363636,2744.448750,493


In [26]:
stat_cols = []
stat_cols_all = []
for col in filtered_cols:
    normal_test = False
    healthy = features[ features['numer'] == 11][col].values
    sick = features[ features['numer'] == 12][col].values
    #sick=np.concatenate((sick, sick), axis=0)
    #_, pval = stats.normaltest(healthy)
    #_, pval2 = stats.normaltest(sick)
    pval=1
    pval2=1
    if pval < 0.05 and pval2 < 0.05:
#         print('normal', col)
        _, pval = stats.ttest_ind(healthy, sick, equal_var=False)
        stat_cols_all.append([col,round(pval,10)])
        if pval < 0.05:
            stat_cols.append([col,round(pval,10)])
            normal_test = False
            
    if not normal_test:    
        #print('not normal test')
        _, pval = stats.ks_2samp(healthy, sick)
        if pval < 0.05:
                stat_cols.append([col,round(pval,10)])
        stat_cols_all.append([col,round(pval,10)])
df_stat = pd.DataFrame(stat_cols, columns=['feature','p-value'])
df_stat.sort_values(by='p-value',inplace=True)
print(df_stat.shape)
df_stat['feature'].values


df_stat2 = pd.DataFrame(stat_cols_all, columns=['feature','p-value'])
df_stat2.sort_values(by='p-value',inplace=True)
#print(df_stat2.shape)
#df_stat2['feature'].values
df_stat2.head()

(0, 2)


,feature,p-value
0,numer,1.0
1,średnia_pom1,1.0
2,odchylenie std_pom1,1.0
3,mediana_pom1,1.0


In [15]:
#export_excel = df_stat.to_excel (r'C:\Users\perszing\Documents\ola\IPAQ_all_1vs2.xlsx', index = None, header=True) 

export_excel = df_stat2.to_excel (r'C:\Users\perszing\Documents\ola\IPAQ_all_1vs2.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path